### previously - workbook3 ;  in continuation - take preprocessSTG8_d3.csv
### Bert Prep
#### ---- set D3 -  Productivity, Shopping, Finance, Social, Personalization

In [1]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('preprocessSTG8_d3.csv')
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Calendar,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,0,0,...,0,1,0,1,0,0,0,1905,imocci is the new means of communication via a...,1797
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,0,0,...,0,0,0,1,0,0,0,3439,neon d iron tech theme keyboard install our ke...,3099


In [3]:
df.shape

(223308, 22)

In [4]:
df.dropna(subset=['Clean_Description'],inplace=True)

In [5]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [7]:
!pip install transformers

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
import numpy as np

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Record the length of each sequence (after truncating to 512)
lengths = []

print('Tokenizing descriptions...')

# For every sentence...
for sent in df.Clean_Description:

    # Report progress
    if ((len(input_ids) % 20000) == 0):
        print('   Read {:,} descriptions.'.format(len(input_ids)))

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

    # Record the truncated length.
    lengths.append(len(encoded_sent))

print('DONE. ')
print('{:>10,} comments'.format(len(input_ids)))

Token indices sequence length is longer than the specified maximum sequence length for this model (688 > 512). Running this sequence through the model will result in indexing errors


Tokenizing descriptions...
   Read 0 descriptions.
   Read 20,000 descriptions.
   Read 40,000 descriptions.
   Read 60,000 descriptions.
   Read 80,000 descriptions.
   Read 100,000 descriptions.
   Read 120,000 descriptions.
   Read 140,000 descriptions.
   Read 160,000 descriptions.
   Read 180,000 descriptions.
   Read 200,000 descriptions.
   Read 220,000 descriptions.
DONE. 
   223,259 comments


In [10]:
print(lengths)
print(len(lengths))
print(df.shape)

[407, 688, 450, 576, 32, 189, 449, 132, 381, 302, 504, 346, 37, 15, 303, 178, 204, 79, 258, 120, 485, 20, 80, 79, 798, 16, 137, 738, 193, 30, 132, 78, 92, 105, 211, 179, 74, 110, 70, 66, 360, 841, 470, 225, 24, 75, 246, 180, 377, 68, 356, 111, 62, 213, 48, 57, 189, 67, 707, 132, 211, 75, 135, 166, 170, 74, 41, 77, 9, 302, 190, 88, 84, 57, 281, 93, 253, 454, 58, 154, 82, 57, 72, 209, 33, 18, 176, 10, 34, 324, 70, 175, 43, 89, 344, 59, 83, 13, 451, 248, 50, 153, 176, 219, 299, 223, 120, 533, 80, 392, 208, 126, 13, 435, 698, 213, 203, 56, 575, 117, 153, 41, 66, 216, 56, 150, 80, 122, 206, 235, 10, 134, 295, 285, 148, 140, 285, 72, 63, 631, 27, 331, 39, 161, 150, 475, 209, 359, 23, 86, 685, 360, 294, 354, 652, 482, 216, 644, 122, 220, 149, 43, 13, 488, 869, 196, 103, 19, 60, 216, 99, 115, 239, 644, 185, 190, 108, 482, 172, 45, 471, 143, 52, 296, 135, 93, 257, 84, 245, 240, 155, 56, 244, 457, 128, 325, 293, 139, 80, 115, 151, 124, 65, 103, 20, 16, 238, 79, 438, 132, 41, 325, 69, 285, 10, 71

In [11]:
df['token_length'] = lengths
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,0,0,...,1,0,1,0,0,0,1905,imocci is the new means of communication via a...,1797,407
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,0,0,...,0,0,1,0,0,0,3439,neon d iron tech theme keyboard install our ke...,3099,688


In [12]:
print(sum(df['token_length'] < 25))

13389


In [13]:
print(sum(df['token_length'] >= 510))

20756


In [14]:
print('    Min length: {:,} tokens'.format(min(lengths)))
print('    Min length: {:,} tokens'.format(max(lengths)))
print(' Median length: {:,} tokens'.format(np.median(lengths)))

    Min length: 3 tokens
    Min length: 1,842 tokens
 Median length: 157.0 tokens


In [15]:
df.head(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,IMOCCI,com.imocci,Social,0.0,89,False,IMOCCI is the new means of communication via A...,https://www.imocci.com/wp-content/uploads/2018...,0,0,...,1,0,1,0,0,0,1905,imocci is the new means of communication via a...,1797,407
1,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,62433,False,💌<b>Neon 3d Iron Tech Theme Keyboard</b>💌\r\n\...,http://bit.ly/EmojiThemePro,0,0,...,0,0,1,0,0,0,3439,neon d iron tech theme keyboard install our ke...,3099,688
2,"Sudan Flag Wallpaper: Flags, Country HD Images",com.techzit.sudanflagwallpaper,Personalization,0.0,468,False,<b>Sudan</b> mobile application provides below...,https://toolsfairy.com/privacy-policy.html,0,1,...,0,0,1,0,0,0,2607,sudan mobile application provides below list o...,2370,450


In [ ]:
## save the df dataset file - 

In [16]:
df.to_csv("preprocessSTG9_d3.csv",index=False)